In [1]:
import gym
import ptan
import ptan.ignite as ptan_ignite
from datetime import datetime, timedelta
import collections
import warnings
from typing import List, Tuple

import torch
import torch.optim as optim
import torch.multiprocessing as mp

from ignite.engine import Engine
from ignite.metrics import RunningAverage
from ignite.contrib.handlers import tensorboard_logger as tb_logger

from lib import dqn_model, common, atari_wrappers

BATCH_MUL = 4
NAME = "04_new_wrappers_parallel"

EpisodeEnded = collections.namedtuple(
    'EpisodeEnded', field_names=('reward', 'steps', 'epsilon'))

In [2]:
def play_func(params, net, cuda, exp_queue):
    env = atari_wrappers.make_atari(params.env_name, skip_noop=True,
                                    skip_maxskip=True)
    env = atari_wrappers.wrap_deepmind(env, pytorch_img=True,
                                       frame_stack=True,
                                       frame_stack_count=2)
    env.seed(common.SEED)
    device = torch.device("cuda" if cuda else "cpu")

    selector = ptan.actions.EpsilonGreedyActionSelector(
        epsilon=params.epsilon_start)
    epsilon_tracker = common.EpsilonTracker(selector, params)
    agent = ptan.agent.DQNAgent(net, selector, device=device)
    exp_source = ptan.experience.ExperienceSourceFirstLast(
        env, agent, gamma=params.gamma)

    for frame_idx, exp in enumerate(exp_source):
        epsilon_tracker.frame(frame_idx/BATCH_MUL)
        exp_queue.put(exp)
        for reward, steps in exp_source.pop_rewards_steps():
            exp_queue.put(EpisodeEnded(reward, steps, selector.epsilon))

In [3]:
class BatchGenerator:
    def __init__(self, buffer: ptan.experience.ExperienceReplayBuffer,
                 exp_queue: mp.Queue,
                 fps_handler: ptan_ignite.EpisodeFPSHandler,
                 initial: int, batch_size: int):
        self.buffer = buffer
        self.exp_queue = exp_queue
        self.fps_handler = fps_handler
        self.initial = initial
        self.batch_size = batch_size
        self._rewards_steps = []
        self.epsilon = None

    def pop_rewards_steps(self) -> List[Tuple[float, int]]:
        res = list(self._rewards_steps)
        self._rewards_steps.clear()
        return res

    def __iter__(self):
        while True:
            while exp_queue.qsize() > 0:
                exp = exp_queue.get()
                if isinstance(exp, EpisodeEnded):
                    self._rewards_steps.append((exp.reward, exp.steps))
                    self.epsilon = exp.epsilon
                else:
                    self.buffer._add(exp)
                    self.fps_handler.step()
            if len(self.buffer) < self.initial:
                continue
            yield self.buffer.sample(self.batch_size * BATCH_MUL)

In [4]:
#getting rid of missing metrics warning
warnings.simplefilter("ignore", category=UserWarning)

mp.set_start_method('spawn')
params = common.HYPERPARAMS['pong']
#parser = argparse.ArgumentParser()
#args = parser.parse_args()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = atari_wrappers.make_atari(params.env_name, skip_noop=True,
                               skip_maxskip=True)
env = atari_wrappers.wrap_deepmind(env, pytorch_img=True,
                                  frame_stack=True,
                                  frame_stack_count=2)

In [5]:
net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)
tgt_net = ptan.agent.TargetNet(net)

buffer = ptan.experience.ExperienceReplayBuffer(
    experience_source=None, buffer_size=params.replay_size)
optimizer = optim.Adam(net.parameters(), lr=params.learning_rate)

# start subprocess and experience queue
exp_queue = mp.Queue(maxsize=BATCH_MUL*2)
play_proc = mp.Process(target=play_func, args=(params, net, True,
                                               exp_queue))
play_proc.start()
fps_handler = ptan_ignite.EpisodeFPSHandler()
batch_generator = BatchGenerator(buffer, exp_queue, fps_handler, params.replay_initial, params.batch_size)


In [6]:
def process_batch(engine, batch):
    optimizer.zero_grad()
    loss_v = common.calc_loss_dqn(batch, net, tgt_net.target_model,
                                  gamma=params.gamma, device=device)
    loss_v.backward()
    optimizer.step()
    if engine.state.iteration % params.target_net_sync == 0:
        tgt_net.sync()
    return {
        "loss": loss_v.item(),
        "epsilon": batch_generator.epsilon,
    }

engine = Engine(process_batch)
ptan_ignite.EndOfEpisodeHandler(batch_generator, bound_avg_reward=15.0).attach(engine)
fps_handler.attach(engine, manual_step=True)

In [7]:
@engine.on(ptan_ignite.EpisodeEvents.EPISODE_COMPLETED)
def episode_completed(trainer: Engine):
    print("Episode %d: reward=%s, steps=%s, speed=%.3f frames/s, elapsed=%s" % (
        trainer.state.episode, trainer.state.episode_reward,
        trainer.state.episode_steps, trainer.state.metrics.get('avg_fps', 0),
        timedelta(seconds=trainer.state.metrics.get('time_passed', 0))))
    
@engine.on(ptan_ignite.EpisodeEvents.BOUND_REWARD_REACHED)
def game_solved(trainer: Engine):
    print("Game solved in %s, after %d episodes and %d iterations!" % (
        timedelta(seconds=trainer.state.metrics['time_passed']),
        trainer.state.episode, trainer.state.iteration))
    trainer.should_terminate = True
    
logdir = f"runs/{datetime.now().minute}-{params.run_name}-{NAME}"
tb = tb_logger.TensorboardLogger(log_dir=logdir)
RunningAverage(output_transform=lambda v: v['loss']).attach(engine, "avg_loss")

episode_handler = tb_logger.OutputHandler(tag='episodes', metric_names=['reward', 'steps', 'avg_reward'])
tb.attach(engine, log_handler=episode_handler, event_name=ptan_ignite.EpisodeEvents.EPISODE_COMPLETED)

# writing to tensorboard every 100 iterations
ptan_ignite.PeriodicEvents().attach(engine)
handler = tb_logger.OutputHandler(tag="train", metric_names=['avg_loss', 'avg_fps'],
                                  output_transform=lambda a: a)
tb.attach(engine, log_handler=handler, event_name=ptan_ignite.PeriodEvents.ITERS_100_COMPLETED)

engine.run(batch_generator)
play_proc.kill()
play_proc.join()

Engine run is terminating due to exception: .


KeyboardInterrupt: 